In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install --upgrade torch==2.5.0 torchvision torchaudio \
    pytorch-lightning transformers==4.41.0 scikit-learn pandas==2.2.2 tqdm --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import pandas as pd
print(f"PyTorch version: {torch.__version__}")
print(f"Pandas version: {pd.__version__}")


PyTorch version: 2.5.0+cu124
Pandas version: 2.2.2


In [4]:
import torch
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os
import pytorch_lightning as pl
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn import metrics


In [52]:
import pandas as pd

# Set random seed for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# # Placeholder function for dataset loading (to be completed later)
# def get_dataset(tokenizer, type_path, hparams):
#     """Load the dataset from the correct CSV file."""
#     if type_path == "train":
#         return SarcasmDataset(tokenizer, "Train_Dataset.csv", max_len=hparams.max_seq_length)
#     elif type_path in ["test", "val"]:
#         return SarcasmDataset(tokenizer, "Test_Dataset.csv", max_len=hparams.max_seq_length)

def get_dataset(tokenizer, type_path, hparams):
    file_path = f"/content/{type_path}_Dataset.csv"
    print(f"Loading dataset from: {file_path}")

    df = pd.read_csv(file_path)[["tweet", "sarcastic"]]
    print(df.head())

    dataset = SarcasmDataset(tokenizer, df, hparams.max_len)
    return dataset

In [53]:
set_seed(42)
print("Random seed set to 42")


Random seed set to 42


In [78]:
from torch.utils.data import Dataset

# Define SarcasmDataset class for T5 training
class SarcasmDataset(Dataset):
    def __init__(self, tokenizer, df, max_len=512):
        self.max_len = max_len
        self.df = df
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []


        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {
            "source_ids": source_ids,
            "source_mask": src_mask,
            "target_ids": target_ids,
            "target_mask": target_mask
        }

    def _build(self):
        for index, row in self.df.iterrows():
            line = row['tweet'].strip()
            target = "positive </s>" if row['sarcastic'] == 1 else "negative </s>"

            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [line], max_length=self.max_len, padding="max_length", return_tensors="pt"
            )

            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=2, padding="max_length", return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)





In [79]:
import torch
import pytorch_lightning as pl
from transformers import AdamW, T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score

# Define T5FineTuner class
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()

        # Fix: Convert Namespace to dictionary to avoid KeyError
        self.save_hyperparameters(vars(hparams))

        # Load pre-trained T5 model
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,  # Ensure compatibility with latest transformers
        )

    def _step(self, batch):
        lm_labels = batch["target_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100  # Ignore padding token

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=lm_labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs.loss  # Extract loss
        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {"params": [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": self.hparams["weight_decay"]},
            {"params": [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams["learning_rate"], eps=self.hparams["adam_epsilon"])
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.hparams["warmup_steps"], num_training_steps=self.hparams["num_train_epochs"]
        )
        return [optimizer], [scheduler]


In [80]:
from argparse import Namespace

# Define hyperparameters for training
hparams = Namespace(
    model_name_or_path="t5-base",
    tokenizer_name_or_path="t5-base",
    max_len=512,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=16,
    eval_batch_size=32,
    num_train_epochs=7,
    gradient_accumulation_steps=4,
    n_gpu=1,
)

# Initialize model
model = T5FineTuner(hparams)
print("T5FineTuner initialized successfully!")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5FineTuner initialized successfully!


In [81]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

train_dataset = get_dataset(tokenizer, "Train", hparams)
test_dataset = get_dataset(tokenizer, "Test", hparams)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print(train_dataset[0])  # 查看第一条数据


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:290: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Loading dataset from: /content/Train_Dataset.csv
                                               tweet  sarcastic
0  The only thing I got from college is a caffein...          1
1  I love it when professors draw a big question ...          1
2  Remember the hundred emails from companies whe...          1
3  Today my pop-pop told me I was not “forced” to...          1
4  @VolphanCarol @littlewhitty @mysticalmanatee I...          1
Loading dataset from: /content/Test_Dataset.csv
                                               tweet  sarcastic
0  Size on the the Toulouse team, That pack is mo...          0
1                                           Pinball!          0
2  So the Scottish Government want people to get ...          1
3  villainous pro tip : change the device name on...          0
4                    I would date any of these men 🥺          0


/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:290: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Train dataset size: 6934
Test dataset size: 1400
{'source_ids': tensor([   37,   163,   589,    27,   530,    45,  1900,    19,     3,     9,
        24355,  9189,     1,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,


In [82]:
from torch.utils.data import DataLoader

# # Load tokenizer
# tokenizer = T5Tokenizer.from_pretrained("t5-small")

# # Load dataset (replace with your real dataset later)
# import pandas as pd
# data = {"tweet": ["I love Mondays!", "Oh great, another meeting..."], "sarcastic": [0, 1]}
# df = pd.DataFrame(data)

# # Create dataset
# train_dataset = SarcasmDataset(tokenizer, df, max_len=512)
# val_dataset = SarcasmDataset(tokenizer, df, max_len=512)  # For testing, use the same dataset

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=hparams.train_batch_size, shuffle=True)
val_dataloader = DataLoader(test_dataset, batch_size=hparams.eval_batch_size)


In [83]:
from pytorch_lightning.callbacks import ModelCheckpoint

# Set model checkpoint to save best models
checkpoint_callback = ModelCheckpoint(
    dirpath="./t5_isarcasm",
    filename="checkpoint-{epoch:02d}-{val_loss:.2f}",
    monitor="val_loss",
    mode="min",
    save_top_k=3,  # Keep top 3 models
)

# Define Trainer
trainer = pl.Trainer(
    max_epochs=hparams.num_train_epochs,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1 if torch.cuda.is_available() else "auto",
    precision=16 if torch.cuda.is_available() else 32,
    gradient_clip_val=hparams.gradient_accumulation_steps,
    callbacks=[checkpoint_callback],
)


# Start training
trainer.fit(model, train_dataloader, val_dataloader)


/usr/local/lib/python3.11/dist-packages/lightning_fabric/connector.py:572: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/t5_isarcasm exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=7` reached.


In [84]:
# f1 = compute_f1_score(model, val_dataloader, tokenizer)
# print(f"Final F1 Score on Test Set: {f1:.4f}")


In [85]:
def compute_f1_score(model, dataloader, tokenizer):
    model.eval()
    predictions = []
    references = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["source_ids"].to(model.device)
            attention_mask = batch["source_mask"].to(model.device)

            outputs = model.model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=5,
                use_cache=True
            )

            preds = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=True) for ids in outputs]
            targets = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

            predictions.extend(preds)
            references.extend(targets)

    f1 = f1_score(references, predictions, average="weighted")
    return f1, predictions, references


In [86]:
f1, predictions, references = compute_f1_score(model, val_dataloader, tokenizer)

print("Predictions:", predictions[:10])
print("Targets:", references[:10])
print(f"Final F1 Score on Test Set: {f1:.4f}")


Predictions: ['negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative']
Targets: ['negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive']
Final F1 Score on Test Set: 0.8214


In [87]:
from collections import Counter
print("Target 分布:", Counter(references))
print("Prediction 分布:", Counter(predictions))


Target 分布: Counter({'negative': 1200, 'positive': 200})
Prediction 分布: Counter({'negative': 1176, 'positive': 224})


In [88]:
print("Unique values in references:", set(references))
print("Unique values in predictions:", set(predictions))



Unique values in references: {'negative', 'positive'}
Unique values in predictions: {'negative', 'positive'}


In [89]:
valid_labels = {"positive", "negative"}

filtered_references = [ref for ref, pred in zip(references, predictions) if ref in valid_labels and pred in valid_labels]
filtered_predictions = [pred for ref, pred in zip(references, predictions) if ref in valid_labels and pred in valid_labels]


In [90]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(filtered_references, filtered_predictions, pos_label="positive", average="binary")
recall = recall_score(filtered_references, filtered_predictions, pos_label="positive", average="binary")

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Precision: 0.3750
Recall: 0.4200


In [91]:
from sklearn.metrics import f1_score

f1 = f1_score(filtered_references, filtered_predictions, pos_label="positive", average="binary")
print(f"Final Corrected F1 Score: {f1:.4f}")


Final Corrected F1 Score: 0.3962
